<a href="https://colab.research.google.com/github/Pager07/Tensorflow-Data-and-Deployment/blob/master/course%204/week%201/LegacyAndS3API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
print('Using TF version:' , tf.__version__)

Using TF version: 2.3.0


#Merging Splits with Legacy API

In [5]:
all_splits = tfds.Split.TRAIN + tfds.Split.TEST

ds = tfds.load("mnist:1.*.*", split=all_splits)

print("Number of Records: {:,}".format(len(list(ds))))


local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/1.0.0. Subsequent calls will reuse this data.
Number of Records: 70,000


With the Legacy API, we can use the subsplit method to divide the datasets. In the example below, we divide the training set into four splits by specifying the number of subsplits, with the argument k=4.

In [9]:
s1 ,s2 , s3 , s4 = tfds.Split.TRAIN.subsplit(k=4)

dataset_split_1 = tfds.load('mnist:1.*.*', split=s1)
dataset_split_2 = tfds.load('mnist:1.*.*', split=s2)
dataset_split_3 = tfds.load('mnist:1.*.*', split=s3)
dataset_split_4 = tfds.load('mnist:1.*.*', split=s4)

print(len(list(dataset_split_1)))
print(len(list(dataset_split_2)))
print(len(list(dataset_split_3)))
print(len(list(dataset_split_4)))



15000
15000
15000
15000


We can also perform the same operation, by specifying a percentage slice in the subsplit method instead. In the example below, we divide the training set into four splits by specifying a percentage slice, with tfds.percent.

In [11]:
s1 = tfds.Split.TRAIN.subsplit(tfds.percent[0:25])
s2 = tfds.Split.TRAIN.subsplit(tfds.percent[25:50])
s3= tfds.Split.TRAIN.subsplit(tfds.percent[50:75])
s4= tfds.Split.TRAIN.subsplit(tfds.percent[75:100])

dataset_split_1 = tfds.load("mnist:1.*.*", split=s1)
dataset_split_2 = tfds.load("mnist:1.*.*", split=s2)
dataset_split_3 = tfds.load("mnist:1.*.*", split=s3)
dataset_split_4 = tfds.load("mnist:1.*.*", split=s4)

print(len(list(dataset_split_1)))
print(len(list(dataset_split_2)))
print(len(list(dataset_split_3)))
print(len(list(dataset_split_4)))


15000
15000
15000
15000


#Using the New S3 API

Before using the new S3 API, we must first find out whether the MNIST dataset implements the new S3 API. In the cell below we indicate that we want to use version 3.*.* of the MNIST dataset.

In [15]:
mnist_builder = tfds.builder("mnist:3.*.*")
print(mnist_builder.version.implements(tfds.core.Experiment.S3))

True


In [18]:
train_ds , test_ds = tfds.load('mnist:3.*.*', split=['train', 'test'])
print(len(list(train_ds)))
print(len(list(test_ds)))

60000
10000


In [22]:
combined = tfds.load('mnist:3.*.*', split='train+test')
print(len(list(combined)))

70000


We can also use Python style list slicers to specify the data we want. For example, we can specify that we want to take the first 10,000 records of the train split with the string 'train[:10000]', as shown below:

In [23]:
first10k = tfds.load('mnist', split='train[:10000]')
print(len(list(first10k)))

10000


The S3 API, also allows us to specify the percentage of the data we want to use. For example, we can select the first 20\% of the training set with the string 'train[:20%]', as shown below:

In [24]:
first20p = tfds.load('mnist', split= 'train[:20%]')
print(len(list(first20p)))

12000


Because the slices are string-based we can use loops, like the ones shown below, to slice up the dataset and make some pretty complex splits. For example, the loops below create 10 complimentary validation and training sets (each loop returns a list with 5 data set

In [29]:
val_ds = tfds.load('mnist', split=['train[{}%:{}%]'.format(k,k+20) for k in range(0,100,20)
                                   ])
train_ds = tfds.load('mnist', split= ['train[:{}%]+train[{}%:]'.format(k,k+20) for k in range(0,100,20)])

In [27]:
['train[{}%:{}%]'.format(k,k+20) for k in range(0,100,20)
                                   ]

['train[0%:20%]',
 'train[20%:40%]',
 'train[40%:60%]',
 'train[60%:80%]',
 'train[80%:100%]']

In [28]:
['train[:{}%]+train[{}%:]'.format(k,k+20) for k in range(0,100,20)]

['train[:0%]+train[20%:]',
 'train[:20%]+train[40%:]',
 'train[:40%]+train[60%:]',
 'train[:60%]+train[80%:]',
 'train[:80%]+train[100%:]']

In [30]:
val_ds 

[<DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>]

In [31]:
train_ds

[<DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>]

In [32]:
print(len(list(val_ds)))
print(len(list(train_ds)))

5
5


The S3 API also allows us to compose new datasets by using pieces from different splits. For example, we can create a new dataset from the first 10\% of the test set and the last 80\% of the training set, as shown below.

In [33]:
composed_ds = tfds.load('mnist:3.*.*', split='test[:10%]+train[-80%:]')

print(len(list(composed_ds)))

49000
